<a href="https://colab.research.google.com/github/KevinBarranco24/PF_TI/blob/main/Proyecto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto Final - Topicos de la industría

##Integrantes:
* Gabriel Contreras, gabriel.contreras@alumnos.udg.mx
* Diego Galvez, diego.galvez0661@alumnos.udg.mx
* Kevin Morales, kevin.morales6034@alumnos.udg.mx




# MODULES

In [32]:
import numpy as np
import pandas as pd
import math
import panel as pn
import panel.widgets as pnw

pn.extension('plotly')

import matplotlib.pyplot as plt
import plotly.graph_objects as go


# DATASET

In [33]:
#IMPORTAMOS EL DATASET
kidney_df = pd.read_csv('/content/trasplantesderinon_2014.csv', index_col=0, encoding='latin-1')

kidney_df

,Sexo,Cintura,Peso,Talla,IMC,NoPaciente,CKD-EPI-TFG,MDRD-4-TFG,Fecha del TR,Fecha actual,Tiempo posTR (meses),Tiempo años calculado,Tiempo posTR (años),Edad al tiempo del TR (años),Tipodedonante,Edad Actual
Edad,,,,,,,,,,,,,,,,
29,0,82.0,65.0,1.67,23.31,1,54.261920,50.744175,1/1/1992,15/09/2014,272,22.666667,22,7,Madre,29
69,1,106.0,79.0,1.47,36.56,2,52.730701,53.777750,1/9/1992,15/09/2014,264,22.000000,22,47,Hija,69
80,0,92.0,62.0,1.72,20.96,3,77.487688,81.650500,1/9/1995,15/09/2014,228,19.000000,19,61,Sobrino,80
33,1,96.0,64.0,1.60,25.00,4,76.965485,70.196928,1/2/1996,15/09/2014,223,18.583333,18,15,Amiga,33
42,0,116.0,98.0,1.74,32.37,5,80.442696,75.554995,22/01/1997,15/09/2014,212,17.666667,17,25,Hermana,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,0,85.0,69.0,1.76,22.28,451,78.676606,73.394833,1/1/2013,15/09/2014,20,1.666667,1,20,Madre,21
29,0,91.0,72.5,1.72,24.51,452,89.243304,81.847773,30/01/2013,15/09/2014,20,1.666667,1,28,Amigo,29
20,1,80.0,45.5,1.47,21.06,453,76.865132,70.447171,3/4/2013,15/09/2014,17,1.416667,1,19,Padre,20


# Figures

In [34]:
def update_figure(filtered_data):
  fig = go.Figure()

  fig.add_trace(
    go.Scatter3d(
        x=filtered_data['Peso'],
        y=filtered_data['Talla'],
        z=filtered_data['Sexo'],
        mode='markers',
        marker=dict(
            size=10,
            color=filtered_data['Sexo'].map({0: 'blue', 1: 'red'}), # Revisar la implementacion de color
            opacity=0.7,
            showscale=True
        ),
        name='Pacientes'
    )
  )

  fig.update_layout(
      scene=dict(
          xaxis_title='Peso del Paciente (kg)',
          yaxis_title='Altura del Paciente (cm)',
          zaxis_title='Sexo del Paciente'
      ),
      margin=dict(l=0, r=0, b=0, t=0)
  )
  return fig

In [35]:
# Crear una función para aplicar los filtros a tu DataFrame
def apply_filters(filtered_df, poblacion, peso, altura, sexo):

    if sexo == 'Men':
        filtered_df = filtered_df[filtered_df['Sexo'] == 0]

    elif sexo == 'Women':
        filtered_df = filtered_df[filtered_df['Sexo'] == 1]

    filtered_df = filtered_df[filtered_df['Peso'] <= peso]
    filtered_df = filtered_df[filtered_df['Talla'] <= altura]
    filtered_df = filtered_df.iloc[:poblacion]

    return filtered_df

# VARIABLES

In [36]:
poblacion = pnw.IntSlider(name='Number of patients', width=150, value=200, step=5, start=1, end=455)
peso = pnw.FloatSlider(name='Max weight of patients (kg)', width=150, value=50, step=0.5, start=34, end=117.5)
altura = pnw.FloatInput(name='height of patients (cm)', value=1.65, step=0.01, start=1.30, end=1.92,width=150)
sexo = pnw.RadioButtonGroup(name='Gender', options=['Men','Women', 'Both'])
metrica = pnw.Select(name="Metrics", options=["Tipo Donante", "Edad Transplantado", "Edad Actual"], value="Tipo Donante",width=150)
filtered_global = pd.DataFrame(data=None, columns=kidney_df.columns)

In [37]:
@pn.depends(poblacion, peso, altura, sexo)
def figure_dataset(poblacion=100, peso=50, altura=1.5, sexo=1):
  filtered_data = apply_filters(kidney_df, poblacion, peso, altura, sexo)
  return update_figure(filtered_data)

In [40]:
def get_giving_type(metrics_df):
  giving_type_df = pd.DataFrame(columns=['type', 'average'])
  temp_df = pd.DataFrame([{'type':'Padre', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Padre']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Madre', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Madre']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Abuelo', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Abuelo']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'afectivo', 'average':metrics_df[metrics_df['Tipodedonante'] == 'afectivo']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Amigo', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Amigo']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Amiga', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Amiga']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Cadáver', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Cadáver']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Concuño', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Concuño']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Cuñada', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Cuñada']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Esposa', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Esposa']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Esposo', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Esposo']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Hermana', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Hermana']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Hermano', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Hermano']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Hija', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Hija']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Hijo', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Hijo']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Madrastra', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Madrastra']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Novia', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Novia']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Prima', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Prima']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Primo', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Primo']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Sobrina', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Sobrina']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Sobrino', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Sobrino']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Tía', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Tía']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Tío', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Tío']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'Otros', 'average':metrics_df[metrics_df['Tipodedonante'] == 'Otro']['Tiempo posTR (años)'].mean()}])
  giving_type_df = pd.concat([giving_type_df,temp_df], ignore_index=True)
  return giving_type_df

def get_by_age(column_name, metrics_df):
  transplanted_age_df = pd.DataFrame(columns=['type', 'average'])
  temp_df = pd.DataFrame([{'type':'0-10', 'average':metrics_df[(metrics_df[column_name] >= 0) & (metrics_df[column_name] <= 10)]['Tiempo posTR (años)'].mean()}])
  transplanted_age_df = pd.concat([transplanted_age_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'11-20', 'average':metrics_df[(metrics_df[column_name] >= 11) & (metrics_df[column_name] <= 20)]['Tiempo posTR (años)'].mean()}])
  transplanted_age_df = pd.concat([transplanted_age_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'21-30', 'average':metrics_df[(metrics_df[column_name] >= 21) & (metrics_df[column_name] <= 30)]['Tiempo posTR (años)'].mean()}])
  transplanted_age_df = pd.concat([transplanted_age_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'31-40', 'average':metrics_df[(metrics_df[column_name] >= 31) & (metrics_df[column_name] <= 40)]['Tiempo posTR (años)'].mean()}])
  transplanted_age_df = pd.concat([transplanted_age_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'41-50', 'average':metrics_df[(metrics_df[column_name] >= 41) & (metrics_df[column_name] <= 50)]['Tiempo posTR (años)'].mean()}])
  transplanted_age_df = pd.concat([transplanted_age_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'51-60', 'average':metrics_df[(metrics_df[column_name] >= 51) & (metrics_df[column_name] <= 60)]['Tiempo posTR (años)'].mean()}])
  transplanted_age_df = pd.concat([transplanted_age_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'61-70', 'average':metrics_df[(metrics_df[column_name] >= 61) & (metrics_df[column_name] <= 70)]['Tiempo posTR (años)'].mean()}])
  transplanted_age_df = pd.concat([transplanted_age_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'71-80', 'average':metrics_df[(metrics_df[column_name] >= 71) & (metrics_df[column_name] <= 80)]['Tiempo posTR (años)'].mean()}])
  transplanted_age_df = pd.concat([transplanted_age_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'81-90', 'average':metrics_df[(metrics_df[column_name] >= 81) & (metrics_df[column_name] <= 90)]['Tiempo posTR (años)'].mean()}])
  transplanted_age_df = pd.concat([transplanted_age_df,temp_df], ignore_index=True)
  temp_df = pd.DataFrame([{'type':'91-', 'average':metrics_df[metrics_df[column_name] >= 91]['Tiempo posTR (años)'].mean()}])
  transplanted_age_df = pd.concat([transplanted_age_df,temp_df], ignore_index=True)
  return transplanted_age_df

def figure_metrica(df,metrica):

  fig = go.Figure()

  fig.add_trace(go.Bar(x=df.type,y=df.average,name='Promedio'))

  fig.update_layout(scene=dict(xaxis_title='Metrica ' + metrica,yaxis_title='Promedio'))

  return fig

@pn.depends(metrica, poblacion, peso, altura, sexo)
def get_metrics(metrica,poblacion=100, peso=50, altura=1.5, sexo=1):

  df = apply_filters(kidney_df, poblacion, peso, altura, sexo)

  if metrica == 'Tipo Donante':
    df = get_giving_type(kidney_df)
  elif metrica == 'Edad Transplantado':
    df = get_by_age('Edad al tiempo del TR (años)',kidney_df)
  elif metrica == 'Edad Actual':
    df = get_by_age('Edad Actual',kidney_df)

  return figure_metrica(df, metrica)


In [41]:
#INICIAMOS EL PANEL

panel_params = pn.Column(sexo, poblacion, peso, altura, metrica)

pn.Row(panel_params,figure_dataset, get_metrics)

Row
    [0] Column
        [0] RadioButtonGroup(name='Gender', options=['Men', 'Women', 'Both'], value='Men')
        [1] IntSlider(end=455, name='Number of patients', start=1, step=5, value=200, width=150)
        [2] FloatSlider(end=117.5, name='Max weight of p..., start=34, step=0.5, value=50, width=150)
        [3] FloatInput(end=1.92, name='height of patients (cm)', start=1.3, step=0.01, value=1.65, width=150)
        [4] Select(name='Metrics', options=['Tipo Donante', ...], value='Tipo Donante', width=150)
    [1] ParamFunction(function, _pane=Plotly, defer_load=False)
    [2] ParamFunction(function, _pane=Plotly, defer_load=False)